# LIBRARIES

In [22]:
library(httr)
library(jsonlite)
library(tidyverse)
library(dplyr)
library(tidyr)
library(testthat)

In [33]:
library(testthat)

# API KEY

In [2]:
api_key <- Sys.getenv("RAPID-API")
print(api_key)

[1] "c63e714db1msh82da74f7cae7ccbp1aeb67jsnf2684826358a"


# HELPER FUNCTIONS

In [3]:
makeApiRequest <- function(url, queryString, api_key) {
  
  # Make the GET request to the specified URL with given query parameters and headers
  response <- VERB("GET", url, query = queryString,
                         add_headers('X-RapidAPI-Key' = api_key,
                                           'X-RapidAPI-Host' = 'api-football-v1.p.rapidapi.com'),
                         content_type("application/octet-stream"))
  
  # Return the response object for further processing
  json_string <- content(response, "text")
  
  # Convert the JSON string to a dataframe
  df <- fromJSON(json_string, flatten = TRUE)
  df_data <- as.data.frame(df$response)

  return(df_data)
}


In [4]:
# Helper Funtion to print goal_scorer
goal_scorer_dashboard <- function(df) {
  # Sort the data frame by the number of goals in descending order
  top_scorers_sorted <- df %>% arrange(desc(goals.total))

  # Select the top 10 scorers
  top_10_scorers <- head(top_scorers_sorted, 10)

  
  league_name <- top_10_scorers$league.name[1]
  league_country <- top_10_scorers$league.country[1]

  # General League information
  cat("|------------------------|\n")
  cat("League Name:", league_name, "\n")
  cat("League Country:", league_country, "\n")
  cat("|------------------------|\n")

  # Print Top 10 goal Scorers
  for (i in 1:nrow(top_10_scorers)) {
    scorer <- top_10_scorers[i, ]
    cat("Team name:", scorer$team.name, "\n")
    cat("Player:", scorer$player.firstname, scorer$player.lastname, "\n")
    cat("Goals Scored:", scorer$goals.total, "\n")
    cat("|----------------------|\n")
  }
}

# TOP SCORER, TOP ASSISTS

In [5]:
getTopScorers <- function(league, season, api_key) {
  
  # Specific url to request
  url <- "https://api-football-v1.p.rapidapi.com/v3/players/topscorers"

  # Hyperparameters
  queryString <- list(
    league = league,
    season = season
  )

  # Call ApiRequest helper function to obtain dataframe
  df_data <- makeApiRequest(url, queryString, api_key)

  # Unnest Statistics column containing goal scorer data
  df_unnested <- df_data %>% unnest(statistics)
  
  # Select relevant columns from dataframe
  df_unnested <- select(df_unnested, player.firstname, player.lastname, player.nationality, games.position, team.name, 
                        league.name, games.appearences, goals.total, goals.assists, shots.total, shots.on)
  
  # Return the unnested data frame
  return(df_unnested)
}


In [6]:
top_scorers <- getTopScorers(39, 2020, api_key)

No encoding supplied: defaulting to UTF-8.



In [7]:
head(top_scorers)

player.firstname,player.lastname,player.nationality,games.position,team.name,league.name,games.appearences,goals.total,goals.assists,shots.total,shots.on
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>
Harry Edward,Kane,England,Attacker,Tottenham,Premier League,35,23,14,110,53
Mohamed,Salah Hamed Mahrous Ghaly,Egypt,Attacker,Liverpool,Premier League,37,22,5,90,52
Bruno Miguel,Borges Fernandes,Portugal,Midfielder,Manchester United,Premier League,37,18,12,84,51
Heung-Min,Son,Korea Republic,Attacker,Tottenham,Premier League,37,17,10,53,36
Patrick James,Bamford,England,Attacker,Leeds,Premier League,38,17,7,90,47
Dominic Nathaniel,Calvert-Lewin,England,Attacker,Everton,Premier League,33,16,NA,72,46


In [8]:
getTopAssists <- function(league, season, api_key) {
  
  # Specific url to request
  url <- "https://api-football-v1.p.rapidapi.com/v3/players/topassists"

  # Hyperparameters
  queryString <- list(
    league = league,
    season = season
  )

  # Call ApiRequest helper function to obtain dataframe
  df_data <- makeApiRequest(url, queryString, api_key)

  # Unnest Statistics column containing goal scorer data
  df_unnested <- df_data %>% unnest(statistics)
  
  # Select relevant columns from dataframe
  df_unnested <- select(df_unnested, player.firstname, player.lastname, player.nationality, games.position, team.name, league.name,
                        games.appearences, goals.assists, goals.total, passes.total, passes.key, passes.accuracy)
  
  # Return the unnested data frame
  return(df_unnested)
}


In [9]:
top_assists <- getTopAssists(39, 2020, api_key)

No encoding supplied: defaulting to UTF-8.



In [10]:
head(top_assists)

player.firstname,player.lastname,player.nationality,games.position,team.name,league.name,games.appearences,goals.assists,goals.total,passes.total,passes.key,passes.accuracy
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>
Harry Edward,Kane,England,Attacker,Tottenham,Premier League,35,14,23,909,51,18
Bruno Miguel,Borges Fernandes,Portugal,Midfielder,Manchester United,Premier League,37,12,18,2109,97,44
Kevin,De Bruyne,Belgium,Midfielder,Manchester City,Premier League,25,12,6,1263,81,41
Heung-Min,Son,Korea Republic,Attacker,Tottenham,Premier League,37,10,17,1056,76,23
Jack Peter,Grealish,England,Midfielder,Aston Villa,Premier League,26,10,6,996,82,31
Jamie Richard,Vardy,England,Attacker,Leicester,Premier League,34,9,15,434,28,8


# FIXTURES

In [11]:
getFootballFixtures <- function(league, season, api_key) {
  
  # Specific url to request
  url <- "https://api-football-v1.p.rapidapi.com/v3/fixtures"
  
  # Hyperparameters
  queryString <- list(
    league = league,
    season = season
  )
  
  # Call ApiRequest helper function to obtain dataframe
  df_data <- makeApiRequest(url, queryString, api_key)

  # Select relevant columns from dataframe
  df_data <- select(df_data, league.name, fixture.venue.name, fixture.date, 
                           fixture.status.long, teams.home.name, teams.away.name, 
                           score.fulltime.home, score.fulltime.away)
  
  return(df_data)
}


In [12]:
fixtures <- getFootballFixtures(league = 39, season = 2023, api_key)

No encoding supplied: defaulting to UTF-8.



In [13]:
head(fixtures)

,league.name,fixture.venue.name,fixture.date,fixture.status.long,teams.home.name,teams.away.name,score.fulltime.home,score.fulltime.away
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
1,Premier League,Turf Moor,2023-08-11T19:00:00+00:00,Match Finished,Burnley,Manchester City,0,3
2,Premier League,Emirates Stadium,2023-08-12T11:30:00+00:00,Match Finished,Arsenal,Nottingham Forest,2,1
3,Premier League,Vitality Stadium,2023-08-12T14:00:00+00:00,Match Finished,Bournemouth,West Ham,1,1
4,Premier League,The American Express Community Stadium,2023-08-12T14:00:00+00:00,Match Finished,Brighton,Luton,4,1
5,Premier League,Goodison Park,2023-08-12T14:00:00+00:00,Match Finished,Everton,Fulham,0,1
6,Premier League,Bramall Lane,2023-08-12T14:00:00+00:00,Match Finished,Sheffield Utd,Crystal Palace,0,1


# LEAGUE STANDINGS


In [14]:
getFootballStandings <- function(league, season, api_key) {
  
  # Specific url to request
  url <- "https://api-football-v1.p.rapidapi.com/v3/standings"
  
  # Hyperparameters
  queryString <- list(
    league = league,
    season = season
  )
  
  # Call ApiRequest helper function to obtain dataframe
  df_data <- makeApiRequest(url, queryString, api_key)

  # Select relevant columns from dataframe
  df_data <- df_data$league.standings

  # Unlisting twice to obtain df
  inner_list <- df_data[[1]]
  standings_df <- inner_list[[1]]
  
  # Select relevant columns from dataframe
  standings_df <- select(standings_df, rank, team.name, group, description, form, points, all.played,
                         all.win, all.draw, all.lose, all.goals.for, all.goals.against, goalsDiff)

  return(standings_df)
}


In [15]:
standings <- getFootballStandings(39, 2020, api_key)

No encoding supplied: defaulting to UTF-8.



In [16]:
head(standings)

,rank,team.name,group,description,form,points,all.played,all.win,all.draw,all.lose,all.goals.for,all.goals.against,goalsDiff
,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,Manchester City,Premier League,Promotion - Champions League (Group Stage),WLWLW,86,38,27,5,6,83,32,51
2,2,Manchester United,Premier League,Promotion - Champions League (Group Stage),WDLLW,74,38,21,11,6,73,44,29
3,3,Liverpool,Premier League,Promotion - Champions League (Group Stage),WWWWW,69,38,20,9,9,68,42,26
4,4,Chelsea,Premier League,Promotion - Champions League (Group Stage),LWLWW,67,38,19,10,9,58,36,22
5,5,Leicester,Premier League,Promotion - Europa League (Group Stage),LLWLD,66,38,20,6,12,68,50,18
6,6,West Ham,Premier League,Promotion - Europa League (Group Stage),WWDLW,65,38,19,8,11,62,47,15


# TRANSFERS

In [18]:
getTeamTransfers <- function(team, api_key) {
    
    url <- "https://api-football-v1.p.rapidapi.com/v3/transfers"

    queryString <- list(team = "33")

    df_data <- makeApiRequest(url, queryString, api_key)
    
    expanded_df <- unnest(df_data, cols = transfers)

    expanded_df <- expanded_df %>%
    mutate(date = as.Date(date))

    result_df <- expanded_df %>%
    select(-1) %>%
    arrange(desc(date))

    return(result_df)
}

In [19]:
transfers <- getTeamTransfers(team = '33', api_key)

No encoding supplied: defaulting to UTF-8.



In [20]:
transfers

date,type,teams.in.id,teams.in.name,teams.in.logo,teams.out.id,teams.out.name,teams.out.logo,player.id,player.name
<date>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>
2099-09-01,Loan,35,Bournemouth,https://media.api-sports.io/football/teams/35.png,33,Manchester United,https://media.api-sports.io/football/teams/33.png,19609,J. O&apos;Shea
2098-08-01,€ 750K,33,Manchester United,https://media.api-sports.io/football/teams/33.png,1828,York,https://media.api-sports.io/football/teams/1828.png,115964,J. Greening
2094-07-01,Loan,59,Preston,https://media.api-sports.io/football/teams/59.png,33,Manchester United,https://media.api-sports.io/football/teams/33.png,113755,D. Beckham
2024-07-01,N/A,33,Manchester United,https://media.api-sports.io/football/teams/33.png,748,Burton Albion,https://media.api-sports.io/football/teams/748.png,282133,J. Hugill
2024-07-01,N/A,33,Manchester United,https://media.api-sports.io/football/teams/33.png,1351,Port Vale,https://media.api-sports.io/football/teams/1351.png,303010,D. Gore
2024-07-01,N/A,33,Manchester United,https://media.api-sports.io/football/teams/33.png,715,Granada CF,https://media.api-sports.io/football/teams/715.png,70078,F. Pellistri
2024-07-01,N/A,502,Fiorentina,https://media.api-sports.io/football/teams/502.png,33,Manchester United,https://media.api-sports.io/football/teams/33.png,74,S. Amrabat
2024-07-01,N/A,33,Manchester United,https://media.api-sports.io/football/teams/33.png,536,Sevilla,https://media.api-sports.io/football/teams/536.png,180560,H. Mejbri
2024-07-01,N/A,33,Manchester United,https://media.api-sports.io/football/teams/33.png,546,Getafe,https://media.api-sports.io/football/teams/546.png,897,M. Greenwood


## JSON Formatted Tables for Testing

In [2]:
# Make the GET request to the specified URL with given query parameters and headers
response <- VERB("GET", url, query = queryString,
                        add_headers('X-RapidAPI-Key' = api_key,
                                        'X-RapidAPI-Host' = 'api-football-v1.p.rapidapi.com'),
                        content_type("application/octet-stream"))

# Return the response object for further processing
json_string <- content(response, "text")

# Convert the JSON string to a dataframe
df <- fromJSON(json_string, flatten = TRUE)
df_data <- as.data.frame(df$response)


ERROR: Error in VERB("GET", url, query = queryString, add_headers(`X-RapidAPI-Key` = api_key, : could not find function "VERB"


## TOP SCORERS

In [3]:
library(httr)

url <- "https://api-football-v1.p.rapidapi.com/v3/players/topscorers"

queryString <- list(
	league = "39",
	season = "2020"
)

response <- VERB("GET", url, query = queryString, add_headers('X-RapidAPI-Key' = 'c63e714db1msh82da74f7cae7ccbp1aeb67jsnf2684826358a', 'X-RapidAPI-Host' = 'api-football-v1.p.rapidapi.com'), content_type("application/octet-stream"))

content(response, "text")

Warning message:
"package 'httr' was built under R version 4.3.2"
No encoding supplied: defaulting to UTF-8.



[1] "{\"get\":\"players\\/topscorers\",\"parameters\":{\"league\":\"39\",\"season\":\"2020\"},\"errors\":[],\"results\":20,\"paging\":{\"current\":0,\"total\":1},\"response\":[{\"player\":{\"id\":184,\"name\":\"H. Kane\",\"firstname\":\"Harry Edward\",\"lastname\":\"Kane\",\"age\":31,\"birth\":{\"date\":\"1993-07-28\",\"place\":\"London\",\"country\":\"England\"},\"nationality\":\"England\",\"height\":\"188 cm\",\"weight\":\"86 kg\",\"injured\":false,\"photo\":\"https:\\/\\/media.api-sports.io\\/football\\/players\\/184.png\"},\"statistics\":[{\"team\":{\"id\":47,\"name\":\"Tottenham\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/47.png\"},\"league\":{\"id\":39,\"name\":\"Premier League\",\"country\":\"England\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png\",\"flag\":\"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg\",\"season\":2020},\"games\":{\"appearences\":35,\"lineups\":35,\"minutes\":3087,\"number\":null,\"position\":\"Attacker\",\"rating\":\"7.642857\",\"captain\":false},\"substitutes\":{\"in\":0,\"out\":5,\"bench\":0},\"shots\":{\"total\":110,\"on\":53},\"goals\":{\"total\":23,\"conceded\":0,\"assists\":14,\"saves\":null},\"passes\":{\"total\":909,\"key\":51,\"accuracy\":18},\"tackles\":{\"total\":22,\"blocks\":11,\"interceptions\":11},\"duels\":{\"total\":466,\"won\":209},\"dribbles\":{\"attempts\":100,\"success\":52,\"past\":null},\"fouls\":{\"drawn\":60,\"committed\":26},\"cards\":{\"yellow\":1,\"yellowred\":0,\"red\":0},\"penalty\":{\"won\":null,\"commited\":null,\"scored\":4,\"missed\":0,\"saved\":null}}]},{\"player\":{\"id\":306,\"name\":\"Mohamed Salah\",\"firstname\":\"Mohamed\",\"lastname\":\"Salah Hamed Mahrous Ghaly\",\"age\":32,\"birth\":{\"date\":\"1992-06-15\",\"place\":\"Nagrig\",\"country\":\"Egypt\"},\"nationality\":\"Egypt\",\"height\":\"175 cm\",\"weight\":\"71 kg\",\"injured\":false,\"photo\":\"https:\\/\\/media.api-sports.io\\/football\\/players\\/306.png\"},\"statistics\":[{\"team\":{\"id\":40,\"name\":\"Liverpool\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/40.png\"},\"league\":{\"id\":39,\"name\":\"Premier League\",\"country\":\"England\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png\",\"flag\":\"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg\",\"season\":2020},\"games\":{\"appearences\":37,\"lineups\":34,\"minutes\":3082,\"number\":null,\"position\":\"Attacker\",\"rating\":\"7.159459\",\"captain\":false},\"substitutes\":{\"in\":3,\"out\":7,\"bench\":3},\"shots\":{\"total\":90,\"on\":52},\"goals\":{\"total\":22,\"conceded\":0,\"assists\":5,\"saves\":null},\"passes\":{\"total\":1262,\"key\":58,\"accuracy\":28},\"tackles\":{\"total\":18,\"blocks\":null,\"interceptions\":1},\"duels\":{\"total\":297,\"won\":94},\"dribbles\":{\"attempts\":84,\"success\":36,\"past\":null},\"fouls\":{\"drawn\":22,\"committed\":22},\"cards\":{\"yellow\":0,\"yellowred\":0,\"red\":0},\"penalty\":{\"won\":null,\"commited\":null,\"scored\":6,\"missed\":0,\"saved\":null}}]},{\"player\":{\"id\":1485,\"name\":\"Bruno Fernandes\",\"firstname\":\"Bruno Miguel\",\"lastname\":\"Borges Fernandes\",\"age\":30,\"birth\":{\"date\":\"1994-09-08\",\"place\":\"Maia\",\"country\":\"Portugal\"},\"nationality\":\"Portugal\",\"height\":\"179 cm\",\"weight\":\"69 kg\",\"injured\":false,\"photo\":\"https:\\/\\/media.api-sports.io\\/football\\/players\\/1485.png\"},\"statistics\":[{\"team\":{\"id\":33,\"name\":\"Manchester United\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/33.png\"},\"league\":{\"id\":39,\"name\":\"Premier League\",\"country\":\"England\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png\",\"flag\":\"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg\",\"season\":2020},\"games\":{\"appearences\":37,\"lineups\":35,\"minutes\":3109,\"number\":null,\"position\":\"Midfielder\",\"rating\":\"7.564864\",\"captain\":false},\"substitutes\":{\"in\":2,\"out\":12,\"bench\":2},\"shots\":{\"total\":84,

## TOP ASSISTS

In [4]:
library(httr)

url <- "https://api-football-v1.p.rapidapi.com/v3/players/topassists"

queryString <- list(
	league = "39",
	season = "2020"
)

response <- VERB("GET", url, query = queryString, add_headers('X-RapidAPI-Key' = 'c63e714db1msh82da74f7cae7ccbp1aeb67jsnf2684826358a', 'X-RapidAPI-Host' = 'api-football-v1.p.rapidapi.com'), content_type("application/octet-stream"))

content(response, "text")

No encoding supplied: defaulting to UTF-8.



[1] "{\"get\":\"players\\/topassists\",\"parameters\":{\"league\":\"39\",\"season\":\"2020\"},\"errors\":[],\"results\":20,\"paging\":{\"current\":0,\"total\":1},\"response\":[{\"player\":{\"id\":184,\"name\":\"H. Kane\",\"firstname\":\"Harry Edward\",\"lastname\":\"Kane\",\"age\":31,\"birth\":{\"date\":\"1993-07-28\",\"place\":\"London\",\"country\":\"England\"},\"nationality\":\"England\",\"height\":\"188 cm\",\"weight\":\"86 kg\",\"injured\":false,\"photo\":\"https:\\/\\/media.api-sports.io\\/football\\/players\\/184.png\"},\"statistics\":[{\"team\":{\"id\":47,\"name\":\"Tottenham\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/47.png\"},\"league\":{\"id\":39,\"name\":\"Premier League\",\"country\":\"England\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png\",\"flag\":\"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg\",\"season\":2020},\"games\":{\"appearences\":35,\"lineups\":35,\"minutes\":3087,\"number\":null,\"position\":\"Attacker\",\"rating\":\"7.642857\",\"captain\":false},\"substitutes\":{\"in\":0,\"out\":5,\"bench\":0},\"shots\":{\"total\":110,\"on\":53},\"goals\":{\"total\":23,\"conceded\":0,\"assists\":14,\"saves\":null},\"passes\":{\"total\":909,\"key\":51,\"accuracy\":18},\"tackles\":{\"total\":22,\"blocks\":11,\"interceptions\":11},\"duels\":{\"total\":466,\"won\":209},\"dribbles\":{\"attempts\":100,\"success\":52,\"past\":null},\"fouls\":{\"drawn\":60,\"committed\":26},\"cards\":{\"yellow\":1,\"yellowred\":0,\"red\":0},\"penalty\":{\"won\":null,\"commited\":null,\"scored\":4,\"missed\":0,\"saved\":null}}]},{\"player\":{\"id\":1485,\"name\":\"Bruno Fernandes\",\"firstname\":\"Bruno Miguel\",\"lastname\":\"Borges Fernandes\",\"age\":30,\"birth\":{\"date\":\"1994-09-08\",\"place\":\"Maia\",\"country\":\"Portugal\"},\"nationality\":\"Portugal\",\"height\":\"179 cm\",\"weight\":\"69 kg\",\"injured\":false,\"photo\":\"https:\\/\\/media.api-sports.io\\/football\\/players\\/1485.png\"},\"statistics\":[{\"team\":{\"id\":33,\"name\":\"Manchester United\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/33.png\"},\"league\":{\"id\":39,\"name\":\"Premier League\",\"country\":\"England\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png\",\"flag\":\"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg\",\"season\":2020},\"games\":{\"appearences\":37,\"lineups\":35,\"minutes\":3109,\"number\":null,\"position\":\"Midfielder\",\"rating\":\"7.564864\",\"captain\":false},\"substitutes\":{\"in\":2,\"out\":12,\"bench\":2},\"shots\":{\"total\":84,\"on\":51},\"goals\":{\"total\":18,\"conceded\":0,\"assists\":12,\"saves\":null},\"passes\":{\"total\":2109,\"key\":97,\"accuracy\":44},\"tackles\":{\"total\":57,\"blocks\":null,\"interceptions\":26},\"duels\":{\"total\":364,\"won\":147},\"dribbles\":{\"attempts\":45,\"success\":23,\"past\":null},\"fouls\":{\"drawn\":52,\"committed\":43},\"cards\":{\"yellow\":6,\"yellowred\":0,\"red\":0},\"penalty\":{\"won\":null,\"commited\":null,\"scored\":9,\"missed\":1,\"saved\":null}}]},{\"player\":{\"id\":629,\"name\":\"K. De Bruyne\",\"firstname\":\"Kevin\",\"lastname\":\"De Bruyne\",\"age\":33,\"birth\":{\"date\":\"1991-06-28\",\"place\":\"Drongen\",\"country\":\"Belgium\"},\"nationality\":\"Belgium\",\"height\":\"181 cm\",\"weight\":\"68 kg\",\"injured\":false,\"photo\":\"https:\\/\\/media.api-sports.io\\/football\\/players\\/629.png\"},\"statistics\":[{\"team\":{\"id\":50,\"name\":\"Manchester City\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/50.png\"},\"league\":{\"id\":39,\"name\":\"Premier League\",\"country\":\"England\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png\",\"flag\":\"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg\",\"season\":2020},\"games\":{\"appearences\":25,\"lineups\":23,\"minutes\":2001,\"number\":null,\"position\":\"Midfielder\",\"rating\":\"7.784000\",\"captain\":false},\"substitutes\":{\"in\":2,\"out\":6,\"bench\":6},\"shots\":{\"total\":53,\"o

## FIXTURES

In [5]:
library(httr)

url <- "https://api-football-v1.p.rapidapi.com/v3/fixtures"

queryString <- list(
	league = "39",
	season = "2020"
)

response <- VERB("GET", url, query = queryString, add_headers('X-RapidAPI-Key' = 'c63e714db1msh82da74f7cae7ccbp1aeb67jsnf2684826358a', 'X-RapidAPI-Host' = 'api-football-v1.p.rapidapi.com'), content_type("application/octet-stream"))

content(response, "text")

No encoding supplied: defaulting to UTF-8.



[1] "{\"get\":\"fixtures\",\"parameters\":{\"league\":\"39\",\"season\":\"2020\"},\"errors\":[],\"results\":380,\"paging\":{\"current\":1,\"total\":1},\"response\":[{\"fixture\":{\"id\":592141,\"referee\":\"K. Friend\",\"timezone\":\"UTC\",\"date\":\"2021-01-12T20:15:00+00:00\",\"timestamp\":1610482500,\"periods\":{\"first\":1610482500,\"second\":1610486100},\"venue\":{\"id\":512,\"name\":\"Turf Moor\",\"city\":\"Burnley\"},\"status\":{\"long\":\"Match Finished\",\"short\":\"FT\",\"elapsed\":90}},\"league\":{\"id\":39,\"name\":\"Premier League\",\"country\":\"England\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png\",\"flag\":\"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg\",\"season\":2020,\"round\":\"Regular Season - 1\"},\"teams\":{\"home\":{\"id\":44,\"name\":\"Burnley\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/44.png\",\"winner\":false},\"away\":{\"id\":33,\"name\":\"Manchester United\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/33.png\",\"winner\":true}},\"goals\":{\"home\":0,\"away\":1},\"score\":{\"halftime\":{\"home\":0,\"away\":0},\"fulltime\":{\"home\":0,\"away\":1},\"extratime\":{\"home\":null,\"away\":null},\"penalty\":{\"home\":null,\"away\":null}}},{\"fixture\":{\"id\":592142,\"referee\":\"J. Moss\",\"timezone\":\"UTC\",\"date\":\"2020-09-12T14:00:00+00:00\",\"timestamp\":1599919200,\"periods\":{\"first\":1599919200,\"second\":1599922800},\"venue\":{\"id\":525,\"name\":\"Selhurst Park\",\"city\":\"London\"},\"status\":{\"long\":\"Match Finished\",\"short\":\"FT\",\"elapsed\":90}},\"league\":{\"id\":39,\"name\":\"Premier League\",\"country\":\"England\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png\",\"flag\":\"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg\",\"season\":2020,\"round\":\"Regular Season - 1\"},\"teams\":{\"home\":{\"id\":52,\"name\":\"Crystal Palace\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/52.png\",\"winner\":true},\"away\":{\"id\":41,\"name\":\"Southampton\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/41.png\",\"winner\":false}},\"goals\":{\"home\":1,\"away\":0},\"score\":{\"halftime\":{\"home\":1,\"away\":0},\"fulltime\":{\"home\":1,\"away\":0},\"extratime\":{\"home\":null,\"away\":null},\"penalty\":{\"home\":null,\"away\":null}}},{\"fixture\":{\"id\":592143,\"referee\":\"C. Kavanagh\",\"timezone\":\"UTC\",\"date\":\"2020-09-12T11:30:00+00:00\",\"timestamp\":1599910200,\"periods\":{\"first\":1599910200,\"second\":1599913800},\"venue\":{\"id\":535,\"name\":\"Craven Cottage\",\"city\":\"London\"},\"status\":{\"long\":\"Match Finished\",\"short\":\"FT\",\"elapsed\":90}},\"league\":{\"id\":39,\"name\":\"Premier League\",\"country\":\"England\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png\",\"flag\":\"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg\",\"season\":2020,\"round\":\"Regular Season - 1\"},\"teams\":{\"home\":{\"id\":36,\"name\":\"Fulham\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/36.png\",\"winner\":false},\"away\":{\"id\":42,\"name\":\"Arsenal\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/42.png\",\"winner\":true}},\"goals\":{\"home\":0,\"away\":3},\"score\":{\"halftime\":{\"home\":0,\"away\":1},\"fulltime\":{\"home\":0,\"away\":3},\"extratime\":{\"home\":null,\"away\":null},\"penalty\":{\"home\":null,\"away\":null}}},{\"fixture\":{\"id\":592144,\"referee\":\"M. Oliver\",\"timezone\":\"UTC\",\"date\":\"2020-09-12T16:30:00+00:00\",\"timestamp\":1599928200,\"periods\":{\"first\":1599928200,\"second\":1599931800},\"venue\":{\"id\":550,\"name\":\"Anfield\",\"city\":\"Liverpool\"},\"status\":{\"long\":\"Match Finished\",\"short\":\"FT\",\"elapsed\":90}},\"league\":{\"id\":39,\"name\":\"Premier League\",\"country\":\"England\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png\",\"flag\":\"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg\",\"season\":2020,\"ro

## LEAGUE STANDINGS

In [6]:
library(httr)

url <- "https://api-football-v1.p.rapidapi.com/v3/standings"

queryString <- list(
	season = "2020",
	league = "39"
)

response <- VERB("GET", url, query = queryString, add_headers('X-RapidAPI-Key' = 'c63e714db1msh82da74f7cae7ccbp1aeb67jsnf2684826358a', 'X-RapidAPI-Host' = 'api-football-v1.p.rapidapi.com'), content_type("application/octet-stream"))

content(response, "text")

No encoding supplied: defaulting to UTF-8.



[1] "{\"get\":\"standings\",\"parameters\":{\"league\":\"39\",\"season\":\"2020\"},\"errors\":[],\"results\":1,\"paging\":{\"current\":1,\"total\":1},\"response\":[{\"league\":{\"id\":39,\"name\":\"Premier League\",\"country\":\"England\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png\",\"flag\":\"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg\",\"season\":2020,\"standings\":[[{\"rank\":1,\"team\":{\"id\":50,\"name\":\"Manchester City\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/50.png\"},\"points\":86,\"goalsDiff\":51,\"group\":\"Premier League\",\"form\":\"WLWLW\",\"status\":\"same\",\"description\":\"Promotion - Champions League (Group Stage)\",\"all\":{\"played\":38,\"win\":27,\"draw\":5,\"lose\":6,\"goals\":{\"for\":83,\"against\":32}},\"home\":{\"played\":19,\"win\":13,\"draw\":2,\"lose\":4,\"goals\":{\"for\":43,\"against\":17}},\"away\":{\"played\":19,\"win\":14,\"draw\":3,\"lose\":2,\"goals\":{\"for\":40,\"against\":15}},\"update\":\"2021-05-23T00:00:00+00:00\"},{\"rank\":2,\"team\":{\"id\":33,\"name\":\"Manchester United\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/33.png\"},\"points\":74,\"goalsDiff\":29,\"group\":\"Premier League\",\"form\":\"WDLLW\",\"status\":\"same\",\"description\":\"Promotion - Champions League (Group Stage)\",\"all\":{\"played\":38,\"win\":21,\"draw\":11,\"lose\":6,\"goals\":{\"for\":73,\"against\":44}},\"home\":{\"played\":19,\"win\":9,\"draw\":4,\"lose\":6,\"goals\":{\"for\":38,\"against\":28}},\"away\":{\"played\":19,\"win\":12,\"draw\":7,\"lose\":0,\"goals\":{\"for\":35,\"against\":16}},\"update\":\"2021-05-23T00:00:00+00:00\"},{\"rank\":3,\"team\":{\"id\":40,\"name\":\"Liverpool\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/40.png\"},\"points\":69,\"goalsDiff\":26,\"group\":\"Premier League\",\"form\":\"WWWWW\",\"status\":\"same\",\"description\":\"Promotion - Champions League (Group Stage)\",\"all\":{\"played\":38,\"win\":20,\"draw\":9,\"lose\":9,\"goals\":{\"for\":68,\"against\":42}},\"home\":{\"played\":19,\"win\":10,\"draw\":3,\"lose\":6,\"goals\":{\"for\":29,\"against\":20}},\"away\":{\"played\":19,\"win\":10,\"draw\":6,\"lose\":3,\"goals\":{\"for\":39,\"against\":22}},\"update\":\"2021-05-23T00:00:00+00:00\"},{\"rank\":4,\"team\":{\"id\":49,\"name\":\"Chelsea\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/49.png\"},\"points\":67,\"goalsDiff\":22,\"group\":\"Premier League\",\"form\":\"LWLWW\",\"status\":\"same\",\"description\":\"Promotion - Champions League (Group Stage)\",\"all\":{\"played\":38,\"win\":19,\"draw\":10,\"lose\":9,\"goals\":{\"for\":58,\"against\":36}},\"home\":{\"played\":19,\"win\":9,\"draw\":6,\"lose\":4,\"goals\":{\"for\":31,\"against\":18}},\"away\":{\"played\":19,\"win\":10,\"draw\":4,\"lose\":5,\"goals\":{\"for\":27,\"against\":18}},\"update\":\"2021-05-23T00:00:00+00:00\"},{\"rank\":5,\"team\":{\"id\":46,\"name\":\"Leicester\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/46.png\"},\"points\":66,\"goalsDiff\":18,\"group\":\"Premier League\",\"form\":\"LLWLD\",\"status\":\"same\",\"description\":\"Promotion - Europa League (Group Stage)\",\"all\":{\"played\":38,\"win\":20,\"draw\":6,\"lose\":12,\"goals\":{\"for\":68,\"against\":50}},\"home\":{\"played\":19,\"win\":9,\"draw\":1,\"lose\":9,\"goals\":{\"for\":34,\"against\":30}},\"away\":{\"played\":19,\"win\":11,\"draw\":5,\"lose\":3,\"goals\":{\"for\":34,\"against\":20}},\"update\":\"2021-05-23T00:00:00+00:00\"},{\"rank\":6,\"team\":{\"id\":48,\"name\":\"West Ham\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/48.png\"},\"points\":65,\"goalsDiff\":15,\"group\":\"Premier League\",\"form\":\"WWDLW\",\"status\":\"same\",\"description\":\"Promotion - Europa League (Group Stage)\",\"all\":{\"played\":38,\"win\":19,\"draw\":8,\"lose\":11,\"goals\":{\"for\":62,\"against\":47}},\"home\":{\"played\":19,\"win\":10,\"draw\":4,\"lose\":5,\"goals\":{\"for\":32,\"against\":22}},\"aw

## TRANSFERS

In [7]:
library(httr)

url <- "https://api-football-v1.p.rapidapi.com/v3/transfers"

queryString <- list(team = "33")

response <- VERB("GET", url, query = queryString, add_headers('X-RapidAPI-Key' = 'c63e714db1msh82da74f7cae7ccbp1aeb67jsnf2684826358a', 'X-RapidAPI-Host' = 'api-football-v1.p.rapidapi.com'), content_type("application/octet-stream"))

content(response, "text")

No encoding supplied: defaulting to UTF-8.



[1] "{\"get\":\"transfers\",\"parameters\":{\"team\":\"33\"},\"errors\":[],\"results\":227,\"paging\":{\"current\":1,\"total\":1},\"response\":[{\"player\":{\"id\":19285,\"name\":\"L. Steele\"},\"update\":\"2023-04-05T04:02:27+00:00\",\"transfers\":[{\"date\":\"2006-08-10\",\"type\":\"\\u20ac 250K\",\"teams\":{\"in\":{\"id\":60,\"name\":\"West Brom\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/60.png\"},\"out\":{\"id\":33,\"name\":\"Manchester United\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/33.png\"}}}]},{\"player\":{\"id\":231667,\"name\":\"L. Giverin\"},\"update\":\"2023-09-03T13:53:31+00:00\",\"transfers\":[{\"date\":\"2012-08-29\",\"type\":\"Loan\",\"teams\":{\"in\":{\"id\":740,\"name\":\"Antwerp\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/740.png\"},\"out\":{\"id\":33,\"name\":\"Manchester United\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/33.png\"}}}]},{\"player\":{\"id\":115964,\"name\":\"J. Greening\"},\"update\":\"2023-09-05T00:32:22+00:00\",\"transfers\":[{\"date\":\"2001-08-01\",\"type\":\"\\u20ac 3M\",\"teams\":{\"in\":{\"id\":70,\"name\":\"Middlesbrough\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/70.png\"},\"out\":{\"id\":33,\"name\":\"Manchester United\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/33.png\"}}},{\"date\":\"2098-08-01\",\"type\":\"\\u20ac 750K\",\"teams\":{\"in\":{\"id\":33,\"name\":\"Manchester United\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/33.png\"},\"out\":{\"id\":1828,\"name\":\"York\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/1828.png\"}}}]},{\"player\":{\"id\":115983,\"name\":\"O. Gill\"},\"update\":\"2023-09-05T07:57:26+00:00\",\"transfers\":[{\"date\":\"2010-10-01\",\"type\":\"Loan\",\"teams\":{\"in\":{\"id\":1343,\"name\":\"Bradford\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/1343.png\"},\"out\":{\"id\":33,\"name\":\"Manchester United\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/33.png\"}}}]},{\"player\":{\"id\":17749,\"name\":\"Zachary Harry Dearnley\"},\"update\":\"2023-09-05T07:59:01+00:00\",\"transfers\":[{\"date\":\"2020-07-01\",\"type\":\"N\\/A\",\"teams\":{\"in\":{\"id\":1349,\"name\":\"Oldham\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/1349.png\"},\"out\":{\"id\":33,\"name\":\"Manchester United\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/33.png\"}}}]},{\"player\":{\"id\":102410,\"name\":\"Jack Barmby\"},\"update\":\"2023-09-05T20:50:52+00:00\",\"transfers\":[{\"date\":\"2014-07-01\",\"type\":\"Free\",\"teams\":{\"in\":{\"id\":46,\"name\":\"Leicester\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/46.png\"},\"out\":{\"id\":33,\"name\":\"Manchester United\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/33.png\"}}},{\"date\":\"2014-01-23\",\"type\":\"Loan\",\"teams\":{\"in\":{\"id\":1366,\"name\":\"Hartlepool\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/1366.png\"},\"out\":{\"id\":33,\"name\":\"Manchester United\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/33.png\"}}}]},{\"player\":{\"id\":883,\"name\":\"L. Grant\"},\"update\":\"2023-09-06T03:47:14+00:00\",\"transfers\":[{\"date\":\"2018-07-03\",\"type\":\"N\\/A\",\"teams\":{\"in\":{\"id\":33,\"name\":\"Manchester United\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/33.png\"},\"out\":{\"id\":75,\"name\":\"Stoke City\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/75.png\"}}}]},{\"player\":{\"id\":114896,\"name\":\"D. Higginbotham\"},\"update\":\"2023-09-06T04:09:37+00:00\",\"transfers\":[{\"date\":\"2000-08-01\",\"type\":\"\\u20ac 3M\",\"teams\":{\"in\":{\"id\":69,\"name\":\"Derby\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/69.png\"},\"out\":{\"id\":33,\"name\":\"Manchester United\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/t